# E9. Movielens 영화 추천 실습

## 프로젝트 개요

- 머신러닝 어플리케이션 중 가장 상업적인 성공을 거둔 것으로 평가받고 있는 **추천시스템(Recommender System)**에 대해 알아보려고 함
- 어느덧 추천시스템은 우리 생활 깊숙이 들어와 있음
- 유튜브, 넷플릭스 등 동영상 플랫폼이 기가막히게 내가 좋아할 만한 콘텐츠를 찾아서 자동으로 플레이해 주고 있음
- 아마존, 쿠팡 온라인 쇼핑 사이트에서도 내 취향에 맞는 상품추천을 쉽게 접할 수 있음
- 페이스북, 인스타그램 같은 SNS는 말할 것도 없고, 어쩌면 느끼지도 못한 사이에 뉴스나 광고까지도 추천시스템의 원리가 숨어들어 있음
- 이런 추천시스템의 원리를 한 문장으로 줄인다면 **나와 비슷한 다른 사용자들이 좋아하는 것과 비슷한 것을 내게 추천해 준다**일 것
- 하지만 어떤 사용자들이 나와 비슷한지를 어떻게 알 수 있을까?
- 또 어떤 상품이나 정보가 서로 유사한지는 어떻게 알 수 있는 걸까?
- 추천시스템의 기본 원리를 파악해 보고 나만의 추천시스템을 만들 수 있는 멋진 아이디어 떠올려 보기

## 프로젝트 소개

- 추천시스템의 개념과 목적을 이해
- Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 보기
- 영화 시청 기록을 활용하여 비슷한 영화를 찾고 영화를 추천해 봄
- 추천 시스템에서 자주 사용되는 데이터 구조인 CSR Matrix을 익힘
- 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점 익히기
- 새로운 데이터셋으로 직접 추천 모델 만들어 보기
- MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보기

## 데이터셋 소개

#### 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터
- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있음. 'Movielens 1M Dataset' 사용을 권장
- 별점 데이터는 대표적인 explicit 데이터. 하지만, implicit 데이터로 간주하고 테스트해볼 수 있음
- 별점을 **시청횟수**로 해석해서 생각
- 또한, 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외

#### Cloud Storage에 미리 업로드 된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결해 주기
```python
$ mkdir -p ~/aiffel/recommendata_iu/data/ml-1m
$ ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-1m
```

## 목차


In [1]:
# movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
# pd.set_option('display.max_rows', 3883) ##전체 movie 행 다 나오게
# movies

## 1. 데이터 준비와 전처리

- Movielens 데이터는 'rating.dat' 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있음

In [2]:
import pandas as pd
import os
rating_file_path = os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep = '::', names = ratings_cols, engine = 'python', encoding = "ISO-8859-1")
original_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남기기
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'original_data_size: {original_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / original_data_size:.2%}')

original_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 count('시청 횟수')로 바꾸기
ratings.rename(columns = {'rating':'count'}, inplace = True)

In [5]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터 읽어보기
movie_file_path = os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv(movie_file_path, sep = '::', names = cols, engine = 'python', encoding = 'ISO-8859-1')
movies.head(30)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


## 2. 분석해 보기

### ratings에 있는 유니크한 영화 개수

In [7]:
ratings['movie_id'].nunique()

3628

### rating에 있는 유니크한 사용자 수

In [8]:
ratings['user_id'].nunique()

6039

### movies에 나와 있는 유니크한 영화 장르

In [9]:
movies['genre'].nunique()

301

In [10]:
movies['genre'].unique()

array(["Animation|Children's|Comedy", "Adventure|Children's|Fantasy",
       'Comedy|Romance', 'Comedy|Drama', 'Comedy',
       'Action|Crime|Thriller', "Adventure|Children's", 'Action',
       'Action|Adventure|Thriller', 'Comedy|Drama|Romance',
       'Comedy|Horror', "Animation|Children's", 'Drama',
       'Action|Adventure|Romance', 'Drama|Thriller', 'Drama|Romance',
       'Thriller', 'Action|Comedy|Drama', 'Crime|Drama|Thriller',
       'Drama|Sci-Fi', 'Romance', 'Adventure|Sci-Fi', 'Adventure|Romance',
       "Children's|Comedy|Drama", 'Documentary', 'Drama|War',
       'Action|Crime|Drama', 'Action|Adventure', 'Crime|Thriller',
       "Animation|Children's|Musical|Romance", 'Action|Drama|Thriller',
       "Children's|Comedy", 'Drama|Mystery', 'Sci-Fi|Thriller',
       'Action|Comedy|Crime|Horror|Thriller', 'Drama|Musical',
       'Crime|Drama|Romance', 'Adventure|Drama', 'Action|Thriller',
       "Adventure|Children's|Comedy|Musical", 'Action|Drama|War',
       'Action|Adventur

### 가장 인기 있는 영화 30개(인기순)

In [11]:
# movies DF에 ratings DF를 left join
movie_data = pd.merge(ratings, movies, how = 'left', on = 'movie_id')
movie_data.tail()

,user_id,movie_id,count,timestamp,title,genre
836473,6040,1090,3,956715518,Platoon (1986),Drama|War
836474,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War
836475,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama
836476,6040,1096,4,956715648,Sophie's Choice (1982),Drama
836477,6040,1097,4,956715569,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi


In [12]:
# 사용하지 않을 컬럼 'movie_id'(title을 key값으로 활용) -> 제거
# 사용하지 않을 컬럼 'timestamp' -> 제거
movie_data = movie_data.drop(columns = ['movie_id', 'timestamp'])
movie_data.head()

,user_id,count,title,genre
0,1,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,3,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,3,My Fair Lady (1964),Musical|Romance
3,1,4,Erin Brockovich (2000),Drama
4,1,5,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [13]:
# 가장 인기있는 영화 30개(인기순)
movie_count = movie_data.groupby('title')['user_id'].count()
movie_count.sort_values(ascending = False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

### 유저별 몇 개의 영화를 보고 있는지에 대한 통계

In [14]:
user_count = movie_data.groupby('user_id')['title'].count()
user_count

user_id
1        53
2       116
3        46
4        19
5       143
       ... 
6036    708
6037    189
6038     18
6039    119
6040    276
Name: title, Length: 6039, dtype: int64

In [15]:
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

In [16]:
# 유저별 시청횟수 중앙값에 대한 통계
user_median = movie_data.groupby('user_id')['count'].median()
user_median

user_id
1       4.0
2       4.0
3       4.0
4       5.0
5       4.0
       ... 
6036    4.0
6037    4.0
6038    4.0
6039    4.0
6040    4.0
Name: count, Length: 6039, dtype: float64

## 3. 내가 선호하는 영화를 10가지 골라서 rating에 추가해 주기

In [17]:
# 본인이 좋아하는 영화 제목 데이터로 바꿔서 추가. 단 이름은 꼭 데이터셋에 있는 것과 동일하게 맞추기
my_favorite = ['Lord of the Rings, The (1978)', 
               'Matrix, The (1999)', 
               'Mission: Impossible (1996)', 
               'Mission: Impossible 2 (2000)', 
               'Sound of Music, The (1965)',
               'Roman Holiday (1953)',
               'Gone with the Wind (1939)',
               'Star Wars: Episode IV - A New Hope (1977)',
               'Star Wars: Episode VI - Return of the Jedi (1983)',
               'Star Wars: Episode I - The Phantom Menace (1999)'
               ]
my_genre = ["Adventure|Animation|Children's|Sci-Fi",
            'Action|Sci-Fi|Thriller',
            'Action|Adventure|Mystery',
            'Action|Thriller',
            'Musical',
            'Comedy|Romance',
            'Drama|Romance|War',
            'Action|Adventure|Fantasy|Sci-Fi',
            'Action|Adventure|Romance|Sci-Fi|War',
            'Action|Adventure|Fantasy|Sci-Fi'
           ]

# 'jiyoon'이라는 user_id가 위 영화를 5번씩 봤다고 가정
my_movie = pd.DataFrame({'user_id':['jiyoon']*10,
                         'title':my_favorite,
                         'genre':my_genre,
                         'count':[5]*10})

# user_id에 'jiyoon'가 없으면 my_favorite 데이터를 기존 데이터에 추가
if not movie_data.isin({'user_id':['jiyoon']})['user_id'].any():
    movie_data = movie_data.append(my_movie)
    
# 자라 추가되었는지 확인
movie_data.tail(20)

,user_id,count,title,genre
836468,6040,3,Dune (1984),Fantasy|Sci-Fi
836469,6040,5,"Last Temptation of Christ, The (1988)",Drama
836470,6040,5,Saving Private Ryan (1998),Action|Drama|War
836471,6040,4,Monty Python's Life of Brian (1979),Comedy
836472,6040,4,Reservoir Dogs (1992),Crime|Thriller
836473,6040,3,Platoon (1986),Drama|War
836474,6040,5,"Crying Game, The (1992)",Drama|Romance|War
836475,6040,5,Welcome to the Dollhouse (1995),Comedy|Drama
836476,6040,4,Sophie's Choice (1982),Drama
836477,6040,4,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi


## 4. 모델에 활용하기 위한 전처리

- 사람이 태어나면 주민등록번호가, 학교에 가면 출석번호가 있듯이 데이터의 관리를 쉽게 하기 위해 번호를 붙여주고 싶음
- 우리가 다루는 데이터에서는 title, genre, user_id 각각에 번호 붙이고 싶음
- 보통 이런 작업을 **indexing**이라고 함
- 추천시스템, 자연어처리에서 자주 하는 작업들
- (참고) pandas.DataFrame.unique()은 특정 컬럼에 포함된 유니크한 데이크만 모아줌. indexing 작업을 위해 매우 유용

In [18]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = movie_data['user_id'].unique()
title_unique = movie_data['title'].unique()
genre_unique = movie_data['genre'].unique()

# 유저, 아티스트 indexing하는 코드 idx는 index의 약자
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}
genre_to_idx = {v:k for k, v in enumerate(genre_unique)}

In [19]:
# 인덱싱이 잘 되었는지 확인

# 6039 명의 유저 중 마지막으로 추가된 유저이니 6040이 나와야 함
print(user_to_idx['jiyoon'])
print(title_to_idx['Lord of the Rings, The (1978)'])
print(genre_to_idx['Action|Adventure|Fantasy|Sci-Fi'])

6039
926
24


In [20]:
# indexing 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16을 참고

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 보기
# 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN가 될테니 dropna()로 제거
temp_user_data = movie_data['user_id'].map(user_to_idx.get).dropna()
# 모든 row가 정상적으로 인덱싱되었다면
if len(temp_user_data) == len(movie_data):
    print('user_id column indexing OK!!')
    # data['user_id']을 인덱싱된 Series로 교체
    movie_data['user_id'] = temp_user_data
else:
    print('user_id column indexing Fail!!')
    
# title_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱
temp_title_data = movie_data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(movie_data):
    print('title column indexing OK!!')
    movie_data['title'] = temp_title_data
else:
    print('title column indexing Fail!')
    
# genre_to_idx을 통해 genre 컬럼도 동일한 방식으로 인덱싱
temp_genre_data = movie_data['genre'].map(genre_to_idx.get).dropna()
if len(temp_genre_data) == len(movie_data):
    print('genre column indexing OK!!')
    movie_data['genre'] = temp_genre_data
else:
    print('genre column indexing Fail!')
    
movie_data

user_id column indexing OK!!
title column indexing OK!!
genre column indexing OK!!


,user_id,count,title,genre
0,0,5,0,0
1,0,3,1,1
2,0,3,2,2
3,0,4,3,0
4,0,5,4,3
...,...,...,...,...
5,6039,5,588,45
6,6039,5,143,53
7,6039,5,44,24
8,6039,5,64,33


## 사용자의 명시적/암묵적 평가

- Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 보기
- 영화 시청 기록을 활용하여 비슷한 영화를 찾고 좋아할 만한 영화를 추천해 봄
- 추천 시스템에서 자주 사용되는 데이터 구조인 CSR Matrix을 익힘
- 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점 익히기
- 새로운 데이터셋으로 직접 추천 모델 만들어 보기

In [21]:
# 데이터셋에서 1회만 시청한 영화의 비율

# 3회 시청한 영화의 비율을 보는 코드
only_one = movie_data[movie_data['count'] < 4]
one, all_data = len(only_one), len(movie_data)
print(f'{one}, {all_data}')
print(f'Ratio of only_one over all movie_data is {one/all_data:2%}')

261197, 836488
Ratio of only_one over all movie_data is 31.225433%


## 4. CSR matrix 직접 만들어 보기

- CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조
- CSR에 대한 자세한 설명은 다음 링크를 참고
- [Scipy sparse matrix handling](https://lovit.github.io/nlp/machine%20learning/2018/04/09/sparse_mtarix_handling/#csr-matrix)
- [StackOverflow csr_matrix](https://stackoverflow.com/questions/53254104/cant-understand-scipy-sparse-csr-matrix-example/62118005#62118005)

- 우리의 data를 CSR Matrix에 맞게 바꿔보기
- DataFrame을 만드는 방식이 다양하듯 csr_matrix를 [만드는 방법](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html)은 다양하게 있음
- 현재 data와 같은 데이터 구조에 적합한 방식은 위 링크(혹은 아래 사진)에서 4번째로 설명하고 있는 방법
![image.png](attachment:image.png)

- 이전 스텝에서 설명한 Matrix Factorization 모델을 [implicit](https://github.com/benfred/implicit) 패키지를 사용하여 학습해보기
    - 'implicit' 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용한 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지
    - 이 패키지에 구현된 'als(AlternatingLeastSquares) 모델'을 사용할 예정
    - 'Matrix Factorization'에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있음

In [22]:
# 실습 - 위에 설명 토대로 만들어보기
from scipy.sparse import csr_matrix

num_user = movie_data['user_id'].nunique()
num_title = movie_data['title'].nunique()

csr_data = csr_matrix((movie_data['count'], (movie_data.user_id, movie_data.title)), shape = (num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836488 stored elements in Compressed Sparse Row format>

In [23]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분. 학습 내용과는 무관
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
os.environ['MKL_NUM_THREADS'] = '1'

## 5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 보기

- AlternatingLeastSquares 클래스의 __init__ 파라미터 살펴보기
    - 1. factors: 유저와 아이템의 벡터를 몇 차원으로 할 것인지
    - 2. regularization: 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
    - 3. use_gpu: GPU를 사용할 것인지
    - 4. iterations: epochs와 같은 의미. 데이터를 몇 번 반복해서 학습할 것인지
- 1, 4를 늘릴수록 학습데이터를 잘 학습하게 되지만, 과적합의 우려가 있으니 좋은 값을 찾아야 함

In [24]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors = 800, regularization = 0.01, use_gpu = False, iterations = 50, dtype = np.float32)

In [25]:
# als 모델은 input으로 (item x user 꼴의 matrix를 받기 때문에 Transpose해주기)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836488 stored elements in Compressed Sparse Column format>

In [26]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/50 [00:00<?, ?it/s]

- 모델 학습이 끝난 상황에서, 아래 2가지 사항을 살펴보기
    - 1. Jiyoon 벡터와 'Lord of the Rings, The (1978)'의 **벡터를 어떻게 만들고 있는지**
    - 2. 두 벡터를 **곱하면 어떤 값**이 나오는지

In [27]:
jiyoon, Lord_of_the_Rings = user_to_idx['jiyoon'], title_to_idx['Lord of the Rings, The (1978)']
jiyoon_vector, Lord_of_the_Rings_vector = als_model.user_factors[jiyoon], als_model.item_factors[Lord_of_the_Rings]

print('슝=3')

슝=3


In [28]:
jiyoon_vector

array([-0.1839183 ,  0.43249005,  0.055235  , -0.40332624,  0.23932993,
       -0.251123  , -0.12939581, -0.10847098,  0.3785765 ,  0.41265664,
       -0.44864368,  0.52177894, -0.0887714 ,  0.01709129,  0.37222162,
       -0.10494711,  0.21458387,  0.12650062, -0.09736424, -0.12267464,
       -0.02575037, -0.45222494,  0.03414068, -0.06216312,  0.16259709,
       -0.19161905, -0.04896241,  0.0180705 ,  0.05203411, -0.15737382,
        0.11119598, -0.00667049, -0.3003982 ,  0.01176562,  0.48139942,
        0.12755077,  0.03107948, -0.26948813,  0.56348306,  0.23911326,
       -0.29062542, -0.13624476, -0.20490834, -0.14009564,  0.30739084,
        0.07302574,  0.13284875,  0.38438958, -0.03968675,  0.1113116 ,
        0.03220798,  0.23882464,  0.1230025 ,  0.3652983 , -0.02871117,
        0.17922927, -0.41650733, -0.3159428 ,  0.58795136, -0.06339419,
       -0.17682645,  0.1602012 , -0.27514836,  0.07778311, -0.02146344,
       -0.17237508,  0.03661646,  0.1416646 , -0.17948286, -0.10

In [29]:
Lord_of_the_Rings_vector

array([ 1.85434222e-02,  7.53026968e-03,  2.28996333e-02,  6.60029845e-03,
        4.62359982e-03,  4.81942575e-03,  9.33550112e-03, -4.36752848e-03,
        1.42909875e-02,  2.09370255e-03, -5.66500146e-03,  1.06244422e-02,
       -3.65818571e-03,  1.73214599e-02,  1.26659507e-02,  3.62861599e-03,
        1.52434101e-02,  2.75293216e-02,  8.65666755e-03,  3.16186016e-03,
        2.74575576e-02, -6.42956002e-03, -1.81338051e-03, -4.73589217e-03,
        2.87427437e-02, -3.65711137e-04, -1.57045887e-03, -8.95292312e-03,
        1.08849173e-02, -5.35084167e-03,  1.37343095e-03,  3.44346417e-03,
       -1.66002922e-02,  6.53170224e-04,  1.92380250e-02,  6.61880663e-03,
        1.49629842e-02,  3.07853357e-03,  7.52672506e-03, -3.64375883e-03,
       -4.61074669e-04, -1.00420583e-02,  7.40659796e-03,  9.68132913e-03,
       -1.01288138e-02, -3.93235404e-03, -1.12616559e-02,  2.29844898e-02,
        1.12524582e-03,  4.20230255e-03,  1.19447028e-02,  1.72951464e-02,
        1.57797977e-03,  

In [30]:
# jiyoon과 Lord_of_the_Rings를 내적하는 코드
np.dot(jiyoon_vector, Lord_of_the_Rings_vector)

0.76883113

- 1이 나와야 될 것 같은데 0.78 정도가 나옴
- factors를 늘리거나 iterator를 늘려야 할 것
- 이후 진행되는 내용은 일단 이렇게 학습된 모델 사용
- Q9. 만약 이 두 수치를 늘려 1에 가깝게 나왔다면 이 모델이 잘 학습되었다고 볼 수 있을까?
- A9. 아직 검증되지 않았음. 학습 데이터에 대해서 fitting은 되었지만 아직 보지 못한 데이터에 대한, 예를 들어 queen을 선호할지에 대한 예측을 잘할지 못할지는 검증하지 않았기 때문

## 6. 내가 선호하는 10가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도 파악

### 선호하는 영화 리스트에 있는 영화로 선호도 파악

In [31]:
jiyoon, mission_impossible = user_to_idx['jiyoon'], title_to_idx['Mission: Impossible 2 (2000)']
jiyoon_vector, mission_impossible_vector = als_model.user_factors[jiyoon], als_model.item_factors[mission_impossible]

print('슝=3')

슝=3


In [32]:
np.dot(jiyoon_vector, mission_impossible_vector)

0.961501

### 선호하는 영화 리스트에 없는 영화로 선호도 파악

In [33]:
jiyoon, money_train = user_to_idx['jiyoon'], title_to_idx['Money Train (1995)']
jiyoon_vector, money_train_vector = als_model.user_factors[jiyoon], als_model.item_factors[money_train]

print('슝=3')

슝=3


In [34]:
np.dot(jiyoon_vector, money_train_vector)

-0.0041221213

## 7. 내가 좋아하는 영화와 비슷한 영화 추천받아 보기

- 'AlternatingLeastSquares' 클래스에 구현되어 있는 'similar_items' 메서드를 통하여 비슷한 아티스트 앚기

In [51]:
# (movie_title, 유사도) Tuple로 반환
favorite_movie = 'Lord of the Rings, The (1978)'
movie_id = title_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N = 15)
similar_movie

[(926, 0.99999994),
 (3448, 0.40339133),
 (3090, 0.40339074),
 (3603, 0.4031433),
 (2788, 0.40177187),
 (3449, 0.40025333),
 (2791, 0.39768407),
 (3447, 0.39700016),
 (3382, 0.39687335),
 (3011, 0.39672184),
 (3618, 0.3963863),
 (3195, 0.3950951),
 (3545, 0.39460236),
 (3177, 0.39425617),
 (3572, 0.39419943)]

In [52]:
# movie id를 다시 영화 이름으로 매핑시켜주기
# artist_to_idx를 뒤집어, index로부터 artist 이름을 얻는 dict 생성
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_movie]

['Lord of the Rings, The (1978)',
 'Hangmen Also Die (1943)',
 'Fausto (1993)',
 'Snowriders (1996)',
 'Giant Gila Monster, The (1959)',
 'Mamma Roma (1962)',
 'Blood Beach (1981)',
 'Raining Stones (1993)',
 'Grandview, U.S.A. (1984)',
 'Kids of the Round Table (1995)',
 'Nemesis 2: Nebula (1995)',
 'Sticky Fingers of Time, The (1997)',
 'Ten Benny (1997)',
 'Just the Ticket (1999)',
 "I Don't Want to Talk About It (De eso no se habla) (1993)"]

In [55]:
# 위의 코드를 함수로 만들고 확인
def get_similar_movie(movie_name: str):
    movie_id = title_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id, N = 15)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie
print("슝=3")

슝=3


In [58]:
get_similar_movie('Mission: Impossible (1996)')

['Mission: Impossible (1996)',
 'Diamonds (1999)',
 'Beautiful (2000)',
 "Brown's Requiem (1998)",
 'Montana (1998)',
 "Where's Marlowe? (1999)",
 'Digimon: The Movie (2000)',
 'Welcome to Woop-Woop (1997)',
 'One Little Indian (1973)',
 'Sweet Nothing (1995)',
 'Jerry & Tom (1998)',
 'Tough and Deadly (1995)',
 'Full Speed (1996)',
 'Secret Agent, The (1996)',
 'Strike! (a.k.a. All I Wanna Do, The Hairy Bird) (1998)']

In [59]:
get_similar_movie('Star Wars: Episode IV - A New Hope (1977)')

['Star Wars: Episode IV - A New Hope (1977)',
 'Loser (1991)',
 'Ulysses (Ulisse) (1954)',
 'Gay Deceivers, The (1969)',
 'Boys (1996)',
 'Slappy and the Stinkers (1998)',
 'Simon Sez (1999)',
 'Zachariah (1971)',
 'Match, The (1999)',
 'Jerry & Tom (1998)',
 'Sunchaser, The (1996)',
 'On Our Merry Way (1948)',
 'Boys, The (1997)',
 'Second Jungle Book: Mowgli & Baloo, The (1997)',
 'Closer You Get, The (2000)']

## 8. 내가 가장 좋아할 만한 영화들을 추천받아 보기

In [60]:
user = user_to_idx['jiyoon']

# recommend에서는 user*item CSR Matrix 받음
movie_recommended = als_model.recommend(user, csr_data, N = 20, filter_already_liked_items = True)
movie_recommended

[(2045, 0.13515465),
 (1244, 0.12895477),
 (910, 0.12779123),
 (1334, 0.12613024),
 (1378, 0.12333718),
 (603, 0.120343775),
 (961, 0.11866408),
 (1561, 0.109034255),
 (726, 0.10569918),
 (1224, 0.10567024),
 (933, 0.10434912),
 (874, 0.10321961),
 (2200, 0.102724954),
 (920, 0.10133519),
 (549, 0.09691833),
 (903, 0.09589224),
 (1309, 0.09586428),
 (70, 0.09549944),
 (1587, 0.09466808),
 (663, 0.09450415)]

In [62]:
[idx_to_title[i[0]] for i in movie_recommended]

['For Love of the Game (1999)',
 'Sommersby (1993)',
 'Heavy Metal (1981)',
 'To Catch a Thief (1955)',
 'Swiss Family Robinson (1960)',
 'Alien Nation (1988)',
 'Say Anything... (1989)',
 'Object of My Affection, The (1998)',
 'Blair Witch Project, The (1999)',
 'About Last Night... (1986)',
 'Wild Wild West (1999)',
 'Invasion of the Body Snatchers (1956)',
 'Bottle Rocket (1996)',
 'Night of the Comet (1984)',
 'High Noon (1952)',
 'Shaft (2000)',
 "Kelly's Heroes (1970)",
 'Ran (1985)',
 'Quiet Man, The (1952)',
 "Singin' in the Rain (1952)"]

- 'AlternatingLeastSquares' 클래스에 구현된 'explain' 메소드를 사용하면 기록을 남긴 데이터 중 **이 추천에 기여한 정도**를 확인할 수 있음

In [63]:
for_love_of_the_game = title_to_idx['For Love of the Game (1999)']
explain = als_model.explain(user, csr_data, itemid = for_love_of_the_game)

In [64]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('Lord of the Rings, The (1978)', 0.05803270327111559),
 ('Sound of Music, The (1965)', 0.03094664680172436),
 ('Roman Holiday (1953)', 0.023830598271674698),
 ('Gone with the Wind (1939)', 0.018466176744799556),
 ('Star Wars: Episode I - The Phantom Menace (1999)', 0.010362091933639713),
 ('Star Wars: Episode IV - A New Hope (1977)', 0.004045278513402119),
 ('Mission: Impossible (1996)', -7.465593501598464e-05),
 ('Matrix, The (1999)', -0.0029843389094726287),
 ('Mission: Impossible 2 (2000)', -0.003720375833198364),
 ('Star Wars: Episode VI - Return of the Jedi (1983)', -0.004103583130146129)]

- 이 method는 추천한 영화의 점수에 기여한 다른 영화와 기여도(합이 콘텐츠의 점수가 됨)를 반환
- 어떤 영화들이 이 추천에 얼마나 기여하고 있는가